In [6]:
from IPython.display import display, HTML,clear_output
# css = """
# .output {
# flex-direction: row;
# flex-wrap: wrap;
# }
# """

# HTML('<style>{}</style>'.format(css))

In [1]:
from pandasql import sqldf
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [8]:
url = "https://api.bseindia.com/BseIndiaAPI/api/DerivOptionChain_IV/w?Expiry=06+Dec+2024"

# Query parameters
params = {
   "Expiry": "06 Dec 2024",  # Expiry date
   "scrip_cd": "1",          # Scrip code
   "strprice": "0"           # Strike price
}
# Headers to mimic a browser request
headers = {
   "Accept": "application/json, text/plain, */*",
   "Referer": "https://www.bseindia.com/",
   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
# Fetch data from the API
response = requests.get(url, headers=headers, params=params)
# Check if the request was successful
if response.status_code == 200:
   # Parse the JSON response
   data = response.json()
   # Extract the "Table" key data
   table_data = data.get("Table", [])
   if table_data:
       # Convert the data into a Pandas DataFrame
       df = pd.DataFrame(table_data)
       # Display the DataFrame as a table
       print("Option Chain Data:")
       display(df)
   else:
       print("No data found in the 'Table' key.")
else:
   print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

Option Chain Data:


,C_Series_Code,C_Open_Interest,C_Absolute_Change_OI,C_Last_Trd_Price,C_NetChange,C_Vol_Traded,C_BIdQty,C_BidPrice,C_OfferPrice,C_OfferQty,...,Product_Type,p_Series_Code,SCRIP_ID,C_SCRIP_ID,comapny_name,C_comapny_name,UlaValue,Strike_Price1,IV,C_IV
0,SENSEX24D0673200CE,,,,,,,,,,...,Index Option,SENSEX24D0673200PE,BSX,BSX,SENSEX,SENSEX,80845.75,73200.00,39.384435004245600000000000,
1,SENSEX24D0673300CE,,,,,,,,,,...,Index Option,SENSEX24D0673300PE,BSX,BSX,SENSEX,SENSEX,80845.75,73300.00,38.624369050333500000000000,
2,SENSEX24D0673400CE,,,,,,,,,,...,Index Option,SENSEX24D0673400PE,BSX,BSX,SENSEX,SENSEX,80845.75,73400.00,38.976610471357200000000000,
3,SENSEX24D0673500CE,,,,,,,,,,...,Index Option,SENSEX24D0673500PE,BSX,BSX,SENSEX,SENSEX,80845.75,73500.00,37.827888887434700000000000,
4,SENSEX24D0673600CE,,,,,,,,,,...,Index Option,SENSEX24D0673600PE,BSX,BSX,SENSEX,SENSEX,80845.75,73600.00,38.691235707341400000000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,SENSEX24D0684800CE,365,0,2.30,2.25,2497,1300,1.65,2.20,10,...,Index Option,SENSEX24D0684800PE,BSX,BSX,SENSEX,SENSEX,80845.75,84800.00,,20.120690542403200000000000
117,SENSEX24D0684900CE,487,0,1.65,1.60,2356,,,,,...,Index Option,SENSEX24D0684900PE,BSX,BSX,SENSEX,SENSEX,80845.75,84900.00,,19.860289178842300000000000
118,SENSEX24D0685000CE,19422,0,2.20,-0.55,68944,,,,,...,Index Option,SENSEX24D0685000PE,BSX,BSX,SENSEX,SENSEX,80845.75,85000.00,,21.000257347276000000000000
119,SENSEX24D0685100CE,1113,0,2.30,-0.65,9657,,,,,...,Index Option,SENSEX24D0685100PE,BSX,BSX,SENSEX,SENSEX,80845.75,85100.00,,21.454414191493300000000000


In [14]:
params = {
   "Expiry": "06 Dec 2024",
   "scrip_cd": "1",
   "strprice": "0"
}
# Headers for the request
headers = {
   "Accept": "application/json, text/plain, */*",
   "Referer": "https://www.bseindia.com/",
   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
# Fetch data
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
   # Parse JSON data
   data = response.json()
   # Extract option chain table
   table_data = data.get("Table", [])
   if table_data:
       # Create DataFrame
       df = pd.DataFrame(table_data)
       # Split Calls (CE) and Puts (PE)
       calls = df[df["C_Series_Code"].str.endswith("CE")]
       puts = df[df["C_Series_Code"].str.endswith("PE")]
       # Merge on Strike Price
       merged = pd.merge(
           calls, puts,
           on="Strike_Price1",
           suffixes=("_Call", "_Put")
       )
       # Select Relevant Columns
       final_df = merged[
           [
               "C_Series_Code_Call", "C_Last_Trd_Price_Call", "C_Open_Interest_Call",
               "Strike_Price1",
               "C_Series_Code_Put", "C_Last_Trd_Price_Put", "C_Open_Interest_Put"
           ]
       ]
       # Rename Columns
       final_df.columns = [
           "Call Series", "Call Last Price", "Call Open Interest",
           "Strike Price",
           "Put Series", "Put Last Price", "Put Open Interest"
       ]
       # Display Data
       display(final_df)
   else:
       print("No data available in the 'Table' key.")
else:
   print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

,Call Series,Call Last Price,Call Open Interest,Strike Price,Put Series,Put Last Price,Put Open Interest


In [12]:
response.text

'{"Table":[{"C_Series_Code":"SENSEX24D0673200CE","C_Open_Interest":"","C_Absolute_Change_OI":"","C_Last_Trd_Price":"","C_NetChange":"","C_Vol_Traded":"","C_BIdQty":"","C_BidPrice":"","C_OfferPrice":"","C_OfferQty":"","C_Series_Id":"821955","Strike_Price":"73,200.00","ScriP_cd":"1","Series_Id":"821663","BIdQty":"","BidPrice":"","OfferPrice":"","OfferQty":"","Vol_Traded":"12670","Last_Trd_Price":"2.15","NetChange":"-1.25","Absolute_Change_OI":"0","Open_Interest":"6668","Ula_Code":"SENSEX","End_TimeStamp":"06 Dec 2024","Product_Type":"Index Option","p_Series_Code":"SENSEX24D0673200PE","SCRIP_ID":"BSX","C_SCRIP_ID":"BSX","comapny_name":"SENSEX              ","C_comapny_name":"SENSEX              ","UlaValue":"80845.75","Strike_Price1":"73200.00","IV":"39.384435004245600000000000","C_IV":""},{"C_Series_Code":"SENSEX24D0673300CE","C_Open_Interest":"","C_Absolute_Change_OI":"","C_Last_Trd_Price":"","C_NetChange":"","C_Vol_Traded":"","C_BIdQty":"","C_BidPrice":"","C_OfferPrice":"","C_OfferQty"